In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Import Modules
from pprint import pprint
import json
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import re

from datetime import datetime, timedelta
import dill
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import mplcyberpunk
import seaborn as sns
import scipy as sp
import scipy.stats as stats
from typing import List, Dict, Callable

# plt.style.use('ggplot')
plt.style.use("cyberpunk")

import os
os.getcwd()

In [ ]:
# Import Custom Modules
from src.helpers import *

In [ ]:
# Load Time Consuming Scraping Session Variables to Translate to DataFrames  
dill.load_session('dill/notebook_env.db')

# 1: Drunk Elephant - Protini Polypeptide Cream
url_1, items_1, users_1 = item_1, item_info_1, user_info_1

# 2: TATCHA - The Water Cream
url_2, items_2, users_2 = item_2, item_info_2, user_info_2

# 3: BELIF - The True Cream Aqua Bomb
url_3, items_3, users_3 = item_3, item_info_3, user_info_3

# 4: BOSCIA - Cactus Water Moisturizer
url_4, items_4, users_4 = item_4, item_info_4, user_info_4

# 5: OLEHENRIKSEN - C-Rush™ Vita
url_5, items_5, users_5 = item_5, item_info_5, user_info_5

# 6: CLINIQUE - Dramatically Different Moisturizing Gel
url_6, items_6, users_6 = item_6, item_info_6, user_info_6

# 7: ORIGINS - Ginzing Energy Boosting Gel Moisturizer
url_7, items_7, users_7 = item_7, item_info_7, user_info_7

# 8: JOSIE MARAN - 100 Percent Pure Argan Oil
url_8, items_8, users_8 = item_8, item_info_8, user_info_8

# 9: FIRST AID BEAUTY - Ultra Repair® Cream Intense Hydration
url_9, items_9, users_9 = item_9, item_info_9, user_info_9

# 10: Dr. Jart - Cicapair™ Tiger Grass Color Correcting Treatment
url_10, items_10, users_10 = item_10, item_info_10, user_info_10

item_urls = [url_1, url_2, url_3, url_4, url_5, url_6, url_7, url_8, url_9, url_10]
item_collection = [items_1, items_2, items_3, items_4, items_5, items_6, items_7, items_8, items_9, items_10]
user_collection = [users_1, users_2, users_3, users_4, users_5, users_6, users_7, users_8, users_9, users_10]

## Create Item DataFrame

In [ ]:
# Item Structure: item_url, item_brand, item_name, n_loves, avg_stars, ingredients, skin_target, n_reviews, five_stars, four_stars, three_stars, two_stars, one_stars, review_corpus

In [ ]:
# Create item dataframe
item_df = pd.DataFrame({
    'item_url':item_urls,
    'item_brand':[x[0] for x in item_collection],
    'item_name':[x[1] for x in item_collection],
    'n_loves':[x[2] for x in item_collection],
    'avg_stars':[x[3] for x in item_collection], 
    'ingredients':[x[4] for x in item_collection], 
    'skin_target':[x[5] for x in item_collection], 
    'n_reviews':[x[6] for x in item_collection], 
    'star_dist':[x[7] for x in item_collection],
    'review_corpus':[x[8] for x in item_collection]
})

In [ ]:
# Add Granulated star distribution
item_df['five_stars'] = item_df['star_dist'].apply(lambda x: int(x[0].partition(':')[2]))
item_df['four_stars'] = item_df['star_dist'].apply(lambda x: int(x[1].partition(':')[2]))
item_df['three_stars'] = item_df['star_dist'].apply(lambda x: int(x[2].partition(':')[2]))
item_df['two_stars'] = item_df['star_dist'].apply(lambda x: int(x[3].partition(':')[2]))
item_df['one_stars'] = item_df['star_dist'].apply(lambda x: int(x[4].partition(':')[2]))

item_df.drop(columns=['star_dist'], inplace=True)

In [ ]:
item_df.head()

## Create User DataFrame

In [ ]:
# User Structure: item_url, item_brand, item_name ,user_id, user_meta, rating, review
user_collection[0][0] # id, meta, rating, reivew, brand, item_name

In [ ]:
# Create user dataframe
user_df = pd.DataFrame({
    'user_id':[x[0] for product in user_collection for x in product],
    'user_meta':[x[1] for product in user_collection for x in product], 
    'star_rating':[x[2] for product in user_collection for x in product], 
    'review':[x[3] for product in user_collection for x in product],
    'item_brand':[x[4] for product in user_collection for x in product],
    'item_name':[x[5] for product in user_collection for x in product],
})

In [ ]:
# Granulate and Clean User Data

# Cast star rating to int
user_df['star_rating'] = user_df['star_rating'].apply(lambda x: re_max(x, int) if type(x) != int else x)

# Remap items to numbers
item_names = user_df['item_name'].unique().tolist()
name_map = dict(zip(item_names, [f'item_{i}' for i in range(1, len(item_names) + 1)]))
user_df['item_name'] = user_df['item_name'].map(name_map)

# Remap brands to numbers
item_brands = user_df['item_brand'].unique().tolist()
brand_map = dict(zip(item_brands, [f'brand_{i}' for i in range(1, len(item_brands) + 1)]))
user_df['item_brand'] = user_df['item_brand'].map(brand_map)


In [ ]:
# Create dict from user meta info
user_df['user_meta'] = user_df['user_meta'].apply(parse_user_meta)

In [ ]:
# Granulate User Meta
user_df['age'] = user_df['user_meta'].apply(lambda x: x.get('age', None))
user_df['eye_color'] = user_df['user_meta'].apply(lambda x: x.get('eye color', None))
user_df['hair_color'] = user_df['user_meta'].apply(lambda x: x.get('hair color', None))
user_df['skin_tone'] = user_df['user_meta'].apply(lambda x: x.get('skin tone', None))
user_df['skin_type'] = user_df['user_meta'].apply(lambda x: x.get('skin type', None))

user_df.drop(columns=['user_meta'], inplace=True)

In [ ]:
# Create timestamp element 
user_df['timestamp'] = user_df['review'].apply(lambda x: x.split('\n')[0])
# Convert and standardize all timestamps
user_df['timestamp'] = user_df['timestamp'].apply(create_timestamp)

In [ ]:
# Create 'verified' feature
user_df['verified'] = user_df['review'].apply(lambda x: 1 if 'verified' in x.split('\n')[0].lower() else 0)

In [ ]:
user_df.info()

In [ ]:
user_df.head()

In [ ]:
user_df['verified'].value_counts()